In [47]:
!pip install Sastrawi --quiet
!pip install tensorflow --quiet

import numpy as np
import pandas as pd

from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.porter import *
import string
import nltk
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from keras_preprocessing.text import Tokenizer
from keras_preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
from sklearn.utils import resample
from sklearn.utils import shuffle
from sklearn.metrics import confusion_matrix,classification_report, accuracy_score
from sklearn.svm import LinearSVC
import re, io, json

nltk.download("popular", quiet=True)

data = pd.read_csv('https://raw.githubusercontent.com/kawishbit/svm-hate-speech-id/main/clean_dataset.csv')
data.dropna(subset=['Tweet'], how='all', inplace=True)
data = data[['Tweet','HS']]
data.head(10)


alay_words = pd.read_csv('https://raw.githubusercontent.com/kawishbit/svm-hate-speech-id/main/alay.csv')
alay_words.head(10)
indonesian_stopwords = pd.read_csv('https://raw.githubusercontent.com/kawishbit/svm-hate-speech-id/main/stopwords.txt')
indonesian_stopwords = indonesian_stopwords.iloc[:, 0].values.tolist()

stopwords = nltk.corpus.stopwords.words("english")

other_exclusions = ["#ff", "ff", "rt"]
stopwords.extend(other_exclusions)
stopwords.extend(indonesian_stopwords)
stemmer = PorterStemmer()
def replace_alay(tweet):
    output = []
    for word in tweet:
      row = alay_words[alay_words.alay == word]
      if row.empty:
        output.append(word)
      else:
        output.append(str(row['replacement'].values[0]))

    return output


def preprocess(tweet):  
    
    # removal of extra spaces
    regex_pat = re.compile(r'\s+')
    tweet_space = tweet.str.replace(r'\s+', ' ', regex=True)

    # removal of @name[mention]
    regex_pat = re.compile(r'@[\w\-]+')
    tweet_name = tweet_space.str.replace(r'@[\w\-]+', '', regex=True)

    # removal of links[https://abc.com]
    giant_url_regex =  re.compile('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|'
            '[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+')
    tweets = tweet_name.str.replace('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|'
            '[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', '', regex=True)
    
    # removal of punctuations and numbers
    punc_remove = tweets.str.replace("[^a-zA-Z]", " ", regex=True)
    # remove whitespace with a single space
    newtweet=punc_remove.str.replace(r'\s+', ' ', regex=True)
    # remove leading and trailing whitespace
    newtweet=newtweet.str.replace(r'^\s+|\s+?$','', regex=True)
    # replace normal numbers with numbr
    newtweet=newtweet.str.replace(r'\d+(\.\d+)?','numbr', regex=True)
    # removal of capitalization
    tweet_lower = newtweet.str.lower()
    
    # tokenizing
    tokenized_tweet = tweet_lower.apply(lambda x: x.split())

    # removal of stopwords
    tokenized_tweet=  tokenized_tweet.apply(lambda x: [item for item in x if item not in stopwords])
    
    # removal of alay words & slangs
    tokenized_tweet = tokenized_tweet.apply(lambda x:  replace_alay(x))

    # stemming of the tweets
    tokenized_tweet = tokenized_tweet.apply(lambda x: [stemmer.stem(i) for i in x])

    newList = []
    for key,value in tokenized_tweet.items():
      tokenized_tweet[key] = ' '.join(value)
      tweets_p= tokenized_tweet
    
    return tweets_p

processed_tweets = preprocess(data['Tweet'])   

X = processed_tweets
print(X.head(10))

tfidf_vectorizer = TfidfVectorizer(ngram_range=(1, 2),max_df=0.75, min_df=5, max_features=10000)

tfidf = tfidf_vectorizer.fit_transform(X)
tfidf
X = tfidf
Y = data['HS'].astype(int)
X_train_tfidf, X_test_tfidf, y_train, y_test = train_test_split(X, Y, random_state=42, test_size=0.2)

support = LinearSVC(random_state=20)
support.fit(X_train_tfidf,y_train)
y_preds = support.predict(X_test_tfidf)
acc3=accuracy_score(y_test,y_preds)
report = classification_report( y_test, y_preds )
print(report)
print("SVM, Accuracy Score:" , acc3)

0    cowok usaha lacak perhati gue lanta remeh perh...
1    telat tau edan sarap gue gaul cigax jifla cal ...
2    kadang pikir percaya tuhan jatuh kali kali kad...
3                                 tau mata sipit lihat
4             kaum cebong kafir lihat dungu dungu haha
5                              bani taplak kawan kawan
6    deklarasi pilih kepala daerah aman anti hoak w...
7    gue selesai watch aldnoah zero kampret karakt ...
8    admin belanja po nak makan ai kepal milo ai ke...
9                                            enak ngew
Name: Tweet, dtype: object
              precision    recall  f1-score   support

           0       0.85      0.86      0.86      1507
           1       0.81      0.79      0.80      1117

    accuracy                           0.83      2624
   macro avg       0.83      0.83      0.83      2624
weighted avg       0.83      0.83      0.83      2624

SVM, Accuracy Score: 0.8323170731707317
